In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec_sys').getOrCreate()
data = spark.read.csv("/FileStore/shared_uploads/sejal@ibm.com/movielens_ratings.csv", inferSchema=True, header=True)

In [0]:
data.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
 3| 1.0| 0|
 5| 2.0| 0|
 9| 4.0| 0|
 11| 1.0| 0|
 12| 2.0| 0|
 15| 1.0| 0|
 17| 1.0| 0|
 19| 1.0| 0|
 21| 1.0| 0|
 23| 1.0| 0|
 26| 3.0| 0|
 27| 1.0| 0|
 28| 1.0| 0|
 29| 1.0| 0|
 30| 1.0| 0|
 31| 1.0| 0|
 34| 1.0| 0|
 37| 1.0| 0|
 41| 2.0| 0|
+-------+------+------+
only showing top 20 rows

In [0]:
data.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

In [0]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
training, test = data.randomSplit([0.8, 0.2])
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [0]:
model = als.fit(training)
predictions = model.transform(test)
predictions.show()

+-------+------+------+----------+
movieId|rating|userId|prediction|
+-------+------+------+----------+
 85| 3.0| 6| 1.0057057|
 85| 5.0| 16| 2.774545|
 85| 1.0| 28| 3.1700542|
 65| 2.0| 3|-1.2154762|
 65| 1.0| 4|0.35875118|
 65| 2.0| 15|0.17211005|
 65| 1.0| 22|-0.4593948|
 65| 1.0| 28|-1.3621392|
 53| 1.0| 12|-1.1252663|
 53| 2.0| 19| 0.6423933|
 53| 3.0| 20| 0.5852946|
 53| 1.0| 25|-0.2080337|
 78| 1.0| 2| 1.3650306|
 78| 1.0| 17| 1.048069|
 78| 1.0| 20|0.99470466|
 78| 1.0| 24| 0.9563717|
 81| 1.0| 3| 0.5516249|
 81| 2.0| 5| 2.0392094|
 81| 3.0| 18| 1.7885779|
 81| 1.0| 21| 0.6879003|
+-------+------+------+----------+
only showing top 20 rows

In [0]:
# formally evaluate this model
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
rmse = evaluator.evaluate(predictions)
print('RMSE')
print(rmse)
# pretty bad, try this on a larger version of movie lens dataset

RMSE
1.7403826568862162

In [0]:
single_user = test.filter(test['userId'] == 11).select(['movieId', 'userId'])
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 10| 11|
 27| 11|
 32| 11|
 36| 11|
 47| 11|
 59| 11|
 71| 11|
 80| 11|
+-------+------+

In [0]:
recommendations = model.transform(single_user)
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
movieId|userId|prediction|
+-------+------+----------+
 71| 11| 3.6632857|
 36| 11| 3.1190605|
 32| 11| 2.7439446|
 47| 11| 2.7196047|
 10| 11| 1.2252172|
 59| 11| 0.3956313|
 80| 11| 0.3051175|
 27| 11|-2.1084187|
+-------+------+----------+